<a href="https://colab.research.google.com/github/Oec8cek/Ai-Understanding/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B2%80%EC%83%89_API_%ED%81%AC%EB%A1%A4%EB%A7%81_0925%EA%B9%8C%EC%A7%80(%ED%86%B5%EA%B8%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import os
import zipfile
import sys
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.request

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap
# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500
Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300
Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.d0u4eNekbd/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.pWVB2MTcHy/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.XHVKivfvxH/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [ ]:
# 드라이버 경로: "C:\Users\USER\Desktop\chromedriver-win64\chromedriver-win64\chromedriver.exe"

In [20]:
!pip install webdriver_manager
!pip install webdriver-manager --upgrade

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from time import sleep
from bs4 import BeautifulSoup
import requests
import re
import os
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm

# Dataframe
import pandas as pd

# 2023.08.02 셀레니움 라이브러리 수정사항 반영
# https://stackoverflow.com/questions/76428561/typeerror-webdriver-init-got-multiple-values-for-argument-options
service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=service, options=options)

In [23]:
# 웹 페이지 URL
base_url = 'https://www.scourt.go.kr/portal/news/NewsListAction.work?gubun=2&page='

# 크롤링할 데이터를 저장할 리스트 초기화
url_li = []

# 1페이지부터 50페이지까지 순회
for page in range(1, 3):
    # 페이지의 URL 조합
    url_full = f'{base_url}{page}'

    # 웹 페이지 열기
    driver.get(url_full)

    # 페이지의 HTML 내용 가져오기
    page_source = driver.page_source

    # BeautifulSoup를 사용하여 HTML 파싱
    bs = BeautifulSoup(page_source, 'html.parser')

    # 원하는 요소를 선택하여 데이터 추출
    A = bs.select('#content > div.contentIn > table > tbody > tr > td.tit > a')

    # 데이터 가공 및 저장
    for j in range(0, len(A)):
        B = A[j].text.split()
        C = B[1:]
        D = ' '.join(C)
        url_li.append(D)

# 크롬 드라이버 종료
# driver.quit()

# 크롤링한 데이터 확인
for i, title in enumerate(url_li, 1):
    print(f"{i}. {title}")

1. 과세기준금액 넘는 재산세 일부 공제...대법 "위법 아냐"
2. 건물 산 소유자...대법 "유치권소멸청구 가능"
3. 전도사도 근로자 맞다"...임금·퇴직금 체불 목사 벌금형
4. '환불불가' 조항 적법... "불공정 약관 아냐"
5. 연쇄살인' 권재찬, 대법 선고...무기징역 확정 되나
6. 돌려차기 사건' 가해자 징역 20년 확정
7. 살인' 이은해 무기징역 확정..공범 조현수 징역 30년
8. 기재해 은행 계좌 만들어도 업무방해죄 '아니다'...대법 첫 판 ...
9. 하청노동자 직접 고용"...대법원 판결
10. 필로폰 검출...대법 "투약 시점 특정 안돼" 파기환송
11. 과세기준금액 넘는 재산세 일부 공제...대법 "위법 아냐"
12. 건물 산 소유자...대법 "유치권소멸청구 가능"
13. 전도사도 근로자 맞다"...임금·퇴직금 체불 목사 벌금형
14. '환불불가' 조항 적법... "불공정 약관 아냐"
15. 연쇄살인' 권재찬, 대법 선고...무기징역 확정 되나
16. 돌려차기 사건' 가해자 징역 20년 확정
17. 살인' 이은해 무기징역 확정..공범 조현수 징역 30년
18. 기재해 은행 계좌 만들어도 업무방해죄 '아니다'...대법 첫 판 ...
19. 하청노동자 직접 고용"...대법원 판결
20. 필로폰 검출...대법 "투약 시점 특정 안돼" 파기환송


In [24]:
df = pd.DataFrame(url_li)
df.rename(columns={0:'기사 제목'}, inplace=True)

In [25]:
df = df.drop_duplicates(subset='기사 제목', keep='first')
df

,기사 제목
0,"과세기준금액 넘는 재산세 일부 공제...대법 ""위법 아냐"""
1,"건물 산 소유자...대법 ""유치권소멸청구 가능"""
2,"전도사도 근로자 맞다""...임금·퇴직금 체불 목사 벌금형"
3,"'환불불가' 조항 적법... ""불공정 약관 아냐"""
4,"연쇄살인' 권재찬, 대법 선고...무기징역 확정 되나"
5,돌려차기 사건' 가해자 징역 20년 확정
6,살인' 이은해 무기징역 확정..공범 조현수 징역 30년
7,기재해 은행 계좌 만들어도 업무방해죄 '아니다'...대법 첫 판 ...
8,"하청노동자 직접 고용""...대법원 판결"
9,"필로폰 검출...대법 ""투약 시점 특정 안돼"" 파기환송"


In [ ]:
!pip install Kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=c9847f651f6cfdc764e83ea171d4be75564ae3d06990e4c7bd83443185755746
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [ ]:
from kiwipiepy import Kiwi
import pandas as pd

kiwi = Kiwi()

In [ ]:
# 품사 필터링 및 중복 제거 함수
def filter_pos(text, target_pos=['NNG']):        # 동사 뺴고 명사만 필터링하자(변경사항)
    tokens = kiwi.analyze(text)[0][0]
    filtered_tokens = [word[0] for word in tokens if word[1] in target_pos]

    # 중복 제거 (순서 유지)
    unique_tokens = []
    for token in filtered_tokens:
        if token not in unique_tokens:
            unique_tokens.append(token)

    return unique_tokens

In [26]:
# '기사 제목' 열에 대해 형태소 분석 및 중복 제거하여 '형태소' 열에 추가
df['제목 키워드'] = df['기사 제목'].apply(filter_pos)
data=df.copy()
insta_df = df.copy()

In [27]:
article_titles = df['기사 제목'].tolist()

In [28]:
df

,기사 제목,제목 키워드
0,"과세기준금액 넘는 재산세 일부 공제...대법 ""위법 아냐""","[과세, 기준, 금액, 재산세, 공제, 대법, 위법]"
1,"건물 산 소유자...대법 ""유치권소멸청구 가능""","[건물, 산, 소유자, 대법, 유치, 소멸, 청구, 가능]"
2,"전도사도 근로자 맞다""...임금·퇴직금 체불 목사 벌금형","[전도사, 근로자, 임금, 퇴직금, 체불, 목사, 벌금형]"
3,"'환불불가' 조항 적법... ""불공정 약관 아냐""","[환불, 불가, 조항, 적법, 공정, 약관]"
4,"연쇄살인' 권재찬, 대법 선고...무기징역 확정 되나","[대법, 선고, 무기, 징역, 확정]"
5,돌려차기 사건' 가해자 징역 20년 확정,"[사건, 가해자, 징역, 확정]"
6,살인' 이은해 무기징역 확정..공범 조현수 징역 30년,"[살인, 은, 무기, 징역, 확정, 공범]"
7,기재해 은행 계좌 만들어도 업무방해죄 '아니다'...대법 첫 판 ...,"[기재, 은행, 계좌, 업무, 방해죄, 대법, 판]"
8,"하청노동자 직접 고용""...대법원 판결","[하청, 노동자, 고용, 법원, 판결]"
9,"필로폰 검출...대법 ""투약 시점 특정 안돼"" 파기환송","[필로폰, 검출, 대법, 투약, 시점, 특정, 파기, 환송]"


In [30]:
# 네이버 검색 API 호출 URL
api_url = "https://openapi.naver.com/v1/search/news.json"

client_id = 'Brt8hUjgWTCMj2uOGshX'
client_secret = 'QzF9tQI3XC'

# API 호출 시 사용할 헤더 정보 설정
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# 각 기사 제목에 대한 검색 결과의 기사 개수를 저장할 리스트
article_counts = []

# 데이터프레임의 각 행을 반복하면서 검색 결과의 기사 개수를 구함
for article_title in df['기사 제목']:
    # API 호출 파라미터 설정
    params = {
        "query": article_title,  # 검색어를 기사 제목으로 설정
        "display": 1,  # 1개의 기사만 가져와 기사 개수를 구함
    }

    # 네이버 검색 API 호출
    response = requests.get(api_url, headers=headers, params=params)

    # API 응답을 JSON 형식으로 파싱
    response_data = response.json()

    # 기사 개수를 구함
    total_articles = response_data.get("total")
    article_counts.append(total_articles)

# 검색 결과의 기사 개수를 데이터프레임에 추가
df['검색 결과의 기사 개수'] = article_counts
df

,기사 제목,제목 키워드,검색 결과의 기사 개수
0,"과세기준금액 넘는 재산세 일부 공제...대법 ""위법 아냐""","[과세, 기준, 금액, 재산세, 공제, 대법, 위법]",None
1,"건물 산 소유자...대법 ""유치권소멸청구 가능""","[건물, 산, 소유자, 대법, 유치, 소멸, 청구, 가능]",None
2,"전도사도 근로자 맞다""...임금·퇴직금 체불 목사 벌금형","[전도사, 근로자, 임금, 퇴직금, 체불, 목사, 벌금형]",None
3,"'환불불가' 조항 적법... ""불공정 약관 아냐""","[환불, 불가, 조항, 적법, 공정, 약관]",None
4,"연쇄살인' 권재찬, 대법 선고...무기징역 확정 되나","[대법, 선고, 무기, 징역, 확정]",None
5,돌려차기 사건' 가해자 징역 20년 확정,"[사건, 가해자, 징역, 확정]",None
6,살인' 이은해 무기징역 확정..공범 조현수 징역 30년,"[살인, 은, 무기, 징역, 확정, 공범]",None
7,기재해 은행 계좌 만들어도 업무방해죄 '아니다'...대법 첫 판 ...,"[기재, 은행, 계좌, 업무, 방해죄, 대법, 판]",None
8,"하청노동자 직접 고용""...대법원 판결","[하청, 노동자, 고용, 법원, 판결]",None
9,"필로폰 검출...대법 ""투약 시점 특정 안돼"" 파기환송","[필로폰, 검출, 대법, 투약, 시점, 특정, 파기, 환송]",None


In [ ]:
sample=df.copy()
df

In [ ]:
# 각 기사 제목에 대한 검색 결과의 첫 번째 기사 URL을 저장할 리스트
first_article_links = []

# 각 기사 제목에 대한 원본 기사 URL을 저장할 리스트
original_article_links = []

# 데이터프레임의 각 행을 반복하면서 기사 URL을 구함
for article_title in df['기사 제목']:
    # API 호출 파라미터 설정
    params = {
        "query": article_title,  # 검색어를 기사 제목으로 설정
        "display": 1,  # 1개의 기사만 가져와 기사 개수를 구함
    }

    # 네이버 검색 API 호출
    response = requests.get(api_url, headers=headers, params=params)

    # API 응답을 JSON 형식으로 파싱
    response_data = response.json()

    # 기사 검색 결과가 있는지 확인
    items = response_data.get("items", [])

    if items:
        # 첫 번째 기사 URL을 구함
        first_article_link = items[0].get("link")
        first_article_links.append(first_article_link)

        # 원본 기사 URL을 구함 (네이버 뉴스 기사의 원본 URL은 없으므로 네이버 뉴스 URL을 사용)
        original_article_links.append(first_article_link)
    else:
        # 검색 결과가 없는 경우, 빈 문자열 추가
        first_article_links.append("")
        original_article_links.append("")

# 'link' 열과 'original link' 열을 데이터프레임에 추가
df['link'] = first_article_links
df['original link'] = original_article_links

In [ ]:
df.head()

,기사 제목,제목 키워드,검색 결과의 기사 개수,link,original link,기사 내용,기사 본문 내용
0,"과세기준금액 넘는 재산세 일부 공제...대법 ""위법 아냐""","[과세, 기준, 금액, 재산세, 공제, 대법, 위법]",1,https://n.news.naver.com/mnews/article/003/001...,https://n.news.naver.com/mnews/article/003/001...,,
1,"건물 산 소유자...대법 ""유치권소멸청구 가능""","[건물, 산, 소유자, 대법, 유치, 소멸, 청구, 가능]",2,https://n.news.naver.com/mnews/article/052/000...,https://n.news.naver.com/mnews/article/052/000...,,
2,"전도사도 근로자 맞다""...임금·퇴직금 체불 목사 벌금형","[전도사, 근로자, 임금, 퇴직금, 체불, 목사, 벌금형]",1,https://n.news.naver.com/mnews/article/028/000...,https://n.news.naver.com/mnews/article/028/000...,,
3,"'환불불가' 조항 적법... ""불공정 약관 아냐""","[환불, 불가, 조항, 적법, 공정, 약관]",1,https://n.news.naver.com/mnews/article/003/001...,https://n.news.naver.com/mnews/article/003/001...,,
4,"연쇄살인' 권재찬, 대법 선고...무기징역 확정 되나","[대법, 선고, 무기, 징역, 확정]",1,https://n.news.naver.com/mnews/article/014/000...,https://n.news.naver.com/mnews/article/014/000...,,


In [ ]:
# '기사 본문 내용' 열 초기화
df['기사 본문 내용'] = ''

# 각 행별로 처리
for idx, row in df.iterrows():
    article_url = row['original link']  # 기사 원본 링크 가져오기

    try:
        # 기사 페이지 열기
        response = requests.get(article_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 기사 본문 내용 추출 (기사 내용이 웹 페이지에 따라 다를 수 있으므로 수정이 필요할 수 있음)
        article_content = soup.find('div', {'class': 'article-body'}).text.strip()  # 예시 클래스명

        # '기사 본문 내용' 열에 추가
        df.at[idx, '기사 본문 내용'] = article_content
    except Exception as e:
        print(f"Error crawling article: {e}")

In [ ]:
import os
import sys
import urllib.request
import json
import re
import pandas as pd

client_id = "Zwdz_lbupo91QAvrtqsz"
client_secret = "ifGZTCfREm"

# 출력 조건 설정, API 제공하는 전체 다 크롤링
display = 10
start = 1
end = 1000
idx = 0

# 초기 데이터프레임 생성: 컬럼명만 있는 빈 데이터프레임
news_df = pd.DataFrame(columns=('title', 'link', 'description', 'originallink', 'pubDate'))

# 기존 데이터프레임 df의 '기사 제목' 열을 'title' 열로 복사
news_df['title'] = df['기사 제목']

# 기존 데이터프레임 df의 '제목 키워드' 열을 'keyword' 열로 복사
news_df['keyword'] = df['제목 키워드']

# 'link', 'number', 'pubDate' 열 추가
news_df['number'] = None

for idx, article_title in enumerate(df['기사 제목']):
    encText = urllib.parse.quote(article_title)  # 검색어를 URL 인코딩

    links = []  # 각 기사 제목에 대한 검색된 기사 링크들을 저장할 리스트

    for start_idx in range(start, end, display):
        url = "https://openapi.naver.com/v1/search/news?query=" + encText \
        + "&display=" + str(display) \
        + "&start=" + str(start_idx)

        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()

        if rescode == 200:
            response_body = response.read()
            response_dict = json.loads(response_body.decode('utf-8'))
            items = response_dict['items']

            for item in items:
                link = item['link']
                links.append(link)  # 검색된 링크를 리스트에 추가

    # 'link' 열에는 links 리스트를 하나의 문자열로 저장
    # df.at[idx, 'link'] = ', '.join(links)

    # 'number' 열에는 링크 개수 할당
    news_df.loc[idx, 'number'] = len(links)

    # 'originallink' 열에는 첫 번째 링크 문자열만 할당
    news_df.at[idx, 'originallink'] = links[0] if links else None

    # 'pubDate' 열에는 첫 번째 링크의 pubDate를 할당
    news_df.at[idx, 'pubDate'] = links[0] if links else None

    # links 초기화
    # links = []

news_df

In [ ]:
import os
import sys
import urllib.request
import json
import re
import pandas as pd
import time

client_id = "Zwdz_lbupo91QAvrtqsz"
client_secret = "ifGZTCfREm"

# 출력 조건 설정, API 제공하는 전체 다 크롤링
display = 10
start = 1
end = 1000
idx = 0

# 초기 데이터프레임 생성: 컬럼명만 있는 빈 데이터프레임
news_df = pd.DataFrame(columns=('title', 'link', 'description', 'originallink', 'pubDate', 'number'))

news_df['number'] = None

for article_title in article_titles:
    encText = urllib.parse.quote(article_title)  # 검색어를 URL 인코딩
    links = []  # 기사 링크를 저장할 리스트 초기화

    for start_idx in range(start, end, display):
        url = "https://openapi.naver.com/v1/search/news?query=" + encText \
        + "&display=" + str(display) \
        + "&start=" + str(start_idx)

        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()

        if rescode == 200:
            response_body = response.read()
            response_dict = json.loads(response_body.decode('utf-8'))
            items = response_dict['items']

            for item in items:
                remove_tag = re.compile('<.*?>')

                title = re.sub(remove_tag, '', item['title'])

                link = item['link']
                links.append(link)  # 검색된 링크를 리스트에 추가

                description = re.sub(remove_tag, '', item['description'])
                originallink = item['originallink']
                pubDate = item['pubDate']
                news_df.loc[idx] = [title, link, description, originallink, pubDate, None]
                links.append(link)  # 검색된 링크를 리스트에 추가
                idx += 1

            # 'number' 열에 리스트의 길이를 저장
            news_df.loc[news_df['title'] == article_title, 'number'] = len(links)

            try:
                # 'originallink' 열에 첫 번째 기사의 URL 저장
                news_df.loc[news_df['title'] == article_title, 'originallink'] = links[0]
            except IndexError:
                # links 리스트가 비어있는 경우, 즉 링크가 하나도 없는 경우에 대한 처리
                news_df.loc[news_df['title'] == article_title, 'originallink'] = None

        else:
            print("Error Code:" + str(rescode))

        # 요청 간격을 설정하여 API 요청 분산
        time.sleep(1)

# 결과 데이터프레임 출력
news_df

items는 네이버 검색 API로부터 받아온 JSON 응답 데이터에서 실제 검색 결과인 뉴스 기사들을 담고 있는 부분입니다. 네이버 검색 API를 사용하여 뉴스를 검색하면, API 응답에는 여러 정보가 들어 있습니다.

- title: 뉴스 기사의 제목
- link: 뉴스 기사의 링크(URL)
- description: 뉴스 기사의 간단한 요약 또는 내용
- originallink: 뉴스 기사가 원본으로 출처된 링크
- pubDate: 뉴스 기사의 발행 날짜

In [ ]:
news_df['summary'] = news_df['description'].apply(filter_pos)
post = news_df['summary']
news_df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def top_5_keywords(document):
    vectorizer = TfidfVectorizer().fit(document)
    vectors = vectorizer.transform(document)
    summed = np.sum(vectors, axis=0)

    word_scores = [(word, summed[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    sorted_keywords = sorted(word_scores, key=lambda x: x[1], reverse=True)

    # 상위 5개의 단어만 반환
    top_keywords = [keyword[0] for keyword in sorted_keywords[:5]]
    return top_keywords


result_keywords = [top_5_keywords(p) for p in post]

In [ ]:
news_df['keywords'] = result_keywords
news_df

In [ ]:
news_link = news_df['originallink']
news_link

# 여기서부턴 코드 돌리기XX

In [ ]:
# 지식인

# 초기 데이터프레임 생성: 컬럼명만 있는 빈 데이터프레임
kin_df = pd.DataFrame(columns=('title', 'link', 'description',
                                'originallink', 'pubDate'))

for article_title in article_titles:
    encText = urllib.parse.quote(article_title)  # 검색어를 URL 인코딩

    for start_idx in range(start, end, display):
        url = "https://openapi.naver.com/v1/search/kin.json?query=" + encText \
        + "&display=" + str(display) \
        + "&start=" + str(start_idx)

        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()

        if rescode == 200:
            response_body = response.read()
            response_dict = json.loads(response_body.decode('utf-8'))
            items = response_dict['items']

            for item in items:
                remove_tag = re.compile('<.*?>')
                title = re.sub(remove_tag, '', item['title'])
                link = item['link']
                description = re.sub(remove_tag, '', item['description'])
                originallink = item['originallink']
                pubDate = item['pubDate']
                kin_df.loc[idx] = [title, link, description, originallink, pubDate]
                idx += 1
        else:
            print("Error Code:" + str(rescode))

kin_df

In [ ]:
# 해당 키워드(ex. 뇌출혈 등)에 따른 블로그 url
kin_df['link']

In [ ]:
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

In [ ]:
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

import time

# 출력 조건 설정, API 제공하는 전체 다 크롤링
display = 10
start = 1
end = 100
idx = 0

# 초기 데이터프레임 생성: 컬럼명만 있는 빈 데이터프레임
news_df = pd.DataFrame(columns=('title', 'link', 'description', 'originallink', 'pubDate'))

for article_title in article_titles:
    encText = urllib.parse.quote(article_title)  # 검색어를 URL 인코딩
    
    for start_idx in range(start, end, display):
        url = "https://openapi.naver.com/v1/search/news?query=" + encText \
        + "&display=" + str(display) \
        + "&start=" + str(start_idx)

        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()

        if rescode == 200:
            response_body = response.read()
            response_dict = json.loads(response_body.decode('utf-8'))
            items = response_dict['items']

            for item in items:
                remove_tag = re.compile('<.*?>')
                title = re.sub(remove_tag, '', item['title'])
                link = item['link']
                description = re.sub(remove_tag, '', item['description'])
                originallink = item['originallink']
                pubDate = item['pubDate']
                news_df.loc[idx] = [title, link, description, originallink, pubDate]
                idx += 1
        else:
            print("Error Code:" + str(rescode))